In [ ]:
##!/usr/bin/env python
"""plot_contour_skill.py

Script produces contour map of the obs/obs uncertainty/model range/model count within obs uncertainty
for different time periods but with national coverage

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Wed May 22 09:01:00 AEST 2019

"""

In [ ]:
# Load packages

#from __future__ import division
import numpy as np
import netCDF4 as nc
import sys
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray
from mpl_toolkits.basemap import Basemap
import common_functions as cf


In [ ]:
# Simulation Details - without CAM radiation

sims=["PHYS_TEST_RA1_PBL1_CU1", "PHYS_TEST_RA1_PBL1_CU16", "PHYS_TEST_RA1_PBL1_CU2", "PHYS_TEST_RA1_PBL1_CU3", 
      "PHYS_TEST_RA1_PBL2_CU1", "PHYS_TEST_RA1_PBL2_CU16", "PHYS_TEST_RA1_PBL2_CU2", "PHYS_TEST_RA1_PBL2_CU3", 
      "PHYS_TEST_RA1_PBL5_CU1", "PHYS_TEST_RA1_PBL5_CU16", "PHYS_TEST_RA1_PBL5_CU2", "PHYS_TEST_RA1_PBL5_CU3", 
#      "PHYS_TEST_RA3_PBL2_CU1", "PHYS_TEST_RA3_PBL2_CU16", "PHYS_TEST_RA3_PBL2_CU2", 
      "PHYS_TEST_RA4_PBL1_CU1", "PHYS_TEST_RA4_PBL1_CU16", "PHYS_TEST_RA4_PBL1_CU2", "PHYS_TEST_RA4_PBL1_CU3", 
      "PHYS_TEST_RA4_PBL2_CU1", "PHYS_TEST_RA4_PBL2_CU16", "PHYS_TEST_RA4_PBL2_CU2", "PHYS_TEST_RA4_PBL2_CU3", 
      "PHYS_TEST_RA4_PBL5_CU1", "PHYS_TEST_RA4_PBL5_CU16", "PHYS_TEST_RA4_PBL5_CU2", "PHYS_TEST_RA4_PBL5_CU3", 
      "PHYS_TEST_RA5_PBL1_CU1", "PHYS_TEST_RA5_PBL1_CU16", "PHYS_TEST_RA5_PBL1_CU2", "PHYS_TEST_RA5_PBL1_CU3", 
      "PHYS_TEST_RA5_PBL2_CU1", "PHYS_TEST_RA5_PBL2_CU16", "PHYS_TEST_RA5_PBL2_CU2", "PHYS_TEST_RA5_PBL2_CU3", 
      "PHYS_TEST_RA5_PBL5_CU1", "PHYS_TEST_RA5_PBL5_CU16", "PHYS_TEST_RA5_PBL5_CU2", "PHYS_TEST_RA5_PBL5_CU3"]
rlabels=["Dudhia / YSU / KF","Dudhia / YSU / NTiedtke","Dudhia / YSU / BMJ","Dudhia / YSU / GF",
         "Dudhia/ MYJ / KF","Dudhia / MYJ / NTiedtke","Dudhia / MYJ / BMJ","Dudhia / MYJ / GF",
         "Dudhia / MYNN / KF","Dudhia / MYNN / NTiedtke","Dudhia / MYNN / BMJ","Dudhia / MYNN / GF",
#         "CAM / MYJ / KF","CAM / MYJ / NTiedtke","CAM / MYJ / BMJ",
         "RRTMG / YSU / KF","RRTMG / YSU / NTiedtke","RRTMG / YSU / BMJ  ","RRTMG / YSU / GF",
         "RRTMG / MYJ / KF","RRTMG / MYJ / NTiedtke","RRTMG / MYJ / BMJ  ","RRTMG / MYJ / GF",
         "RRTMG / MYNN / KF","RRTMG / MYNN / NTiedtke","RRTMG / MYNN / BMJ  ","RRTMG / MYNN / GF",
         "NGoddard / YSU / KF","NGoddard / YSU / NTiedtke","NGoddard / YSU / BMJ  ","NGoddard / YSU / GF",
         "NGoddard / MYJ / KF","NGoddard / MYJ / NTiedtke","NGoddard / MYJ / BMJ  ","NGoddard / MYJ / GF",
         "NGoddard / MYNN / KF","NGoddard / MYNN / NTiedtke","NGoddard / MYNN / BMJ  ","NGoddard / MYNN / GF"]
group="RAD"


In [ ]:
# These simulations seemed to have the smallest biases
sims=["PHYS_TEST_RA4_PBL2_CU16","PHYS_TEST_RA5_PBL1_CU16","PHYS_TEST_RA5_PBL5_CU1","PHYS_TEST_RA4_PBL2_CU2","PHYS_TEST_RA1_PBL2_CU1"]
rlabels=['MYJ / RRTMG / NTiedtke','YSU / NGoddard / NTiedtke','MYNN / NGoddard / KF','MYJ / RRTMG / BMJ','MYJ / Dudhia / KF']
group="TOP_PERFORMERS"
fig_name_prefix='CONTOUR_SKILL_%s' %(group)

In [ ]:
# CAM simulations
sims=["PHYS_TEST_RA3_PBL2_CU1", "PHYS_TEST_RA3_PBL2_CU16", "PHYS_TEST_RA3_PBL2_CU2"]
rlabels=["CAM / MYJ / KF","CAM / MYJ / NTiedtke","CAM / MYJ / BMJ"]
fig_name_prefix='CONTOUR_SKILL_CAM'

In [ ]:
# Define USER input arguments

# Year / Period of interest
syear = np.int('2008')
eyear = np.int('2010')

# Data directory 
datadir = '/g/data/hh5/tmp/WRF-CABLE/AUS44/postprocess/'

nmod = len(sims)
obs = "AWAP"

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
fig_name_prefix='CONTOUR_SKILL_AWAP_MASKED'
if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)

# Landsea mask
mask_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/geo_em.d01.nc'
lis_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/bdy_data/lis_input.d01.nc'

# Variables
variables=['PR','TX','TN']
nvar=len(variables)

# Define region to zoom in on
latN=-43.0
latX=-11.0
lonN=116.0
lonX=155.0

# Define the time periods of interest    
events=["Extreme Hot","Extreme Dry","Extreme Wet"]
ytarget=["2009","2009","2010"]
mtarget=[["01","02"],["03","04","05","06","07","08","09","10","11"],["01","02","03"]]

#events=["Extreme Dry"]
#ytarget=["2009"]
#mtarget=[["03","04","05","06","07","08","09","10","11"]]

#events=["Extreme Hot","Extreme Wet"]
#ytarget=["2009","2010"]
#mtarget=[["01","02"],["01","02","03"]]

nevents = len(events)


In [ ]:
# Load the mask file
filemask    = nc.Dataset(lis_file)
lsmask      = filemask.variables['LANDMASK'][:]
lsmask      = lsmask.astype(int)  
lat2d       = filemask.variables['lat'][:]
lon2d       = filemask.variables['lon'][:]
filemask.close()

In [ ]:
# Load observational data

filename='%s/%s_regrid_daily_%s_%s_mask.nc' %(datadir,obs,syear,eyear)
file = nc.Dataset(filename)
obsprecip = file.variables['awap_precip_mask'][:]
obst2max = file.variables['awap_t2max_mask'][:]
obst2min = file.variables['awap_t2min_mask'][:]
file.close()

# Load the observational error
filename='%s/%s_rmse_regrid_daily_%s_%s_mask.nc' %(datadir,obs,syear,eyear)
file = nc.Dataset(filename)
obspreciprms = file.variables['awap_precip_mask'][:]
obst2maxrms = file.variables['awap_t2max_mask'][:]
obst2minrms = file.variables['awap_t2min_mask'][:]
file.close()

ndays = 822-61
sind = 213+61

# Truncate data to simulation period - removing the spinup aswell
obsprecipT = obsprecip[sind:sind+ndays,:,:]
obst2maxT = obst2max[sind:sind+ndays,:,:]
obst2minT = obst2min[sind:sind+ndays,:,:]
obspreciprmsT = obspreciprms[sind:sind+ndays,:,:]
obst2maxrmsT = obst2maxrms[sind:sind+ndays,:,:]
obst2minrmsT = obst2minrms[sind:sind+ndays,:,:]

# Mask missing values
obsprecipma = np.ma.masked_array(obsprecipT, obsprecipT>=1.e20).filled(np.nan)
obst2maxma  = np.ma.masked_array(obst2maxT, obst2maxT>=1.e20).filled(np.nan)
obst2minma  = np.ma.masked_array(obst2minT, obst2minT>=1.e20).filled(np.nan)
obspreciprmsma = np.ma.masked_array(obspreciprmsT, obspreciprmsT>=1.e20).filled(np.nan)
obst2maxrmsma  = np.ma.masked_array(obst2maxrmsT, obst2maxrmsT>=1.e20).filled(np.nan)
obst2minrmsma  = np.ma.masked_array(obst2minrmsT, obst2minrmsT>=1.e20).filled(np.nan)

# Get lat/lon indices to truncate data to region of interest
bbox = [lonN,lonX, latN, latX]
i0,i1,j0,j1 = cf.bbox2ij(lon2d,lat2d,bbox)


In [ ]:
# Loop through the experiments
for mind,mname in enumerate(sims):

    # Load precip data - excluding 2 month spinup
    filename='%s/%s_WRF_Precip_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    precip = file.variables['wrf_precip_mask'][61:,:,:]
    file.close()

    # Load temperature data - excluding 2 month spinup
    filename='%s/%s_WRFext_Temps_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    t2max = file.variables['T2MAX'][61:,:,:]
    t2min = file.variables['T2MIN'][61:,:,:]
    time = nc.chartostring(file.variables['Times'][61:,:])
    timesplit = np.asarray([time[x].split("_")[0] for x in range(len(time))])
    years = np.asarray([timesplit[x].split("-")[0] for x in range(len(time))])
    months = np.asarray([timesplit[x].split("-")[1] for x in range(len(time))])
    file.close()
    
    if mind == 0:
        dx = lat2d[j0:j1,i0:i1].shape[0]
        dy = lat2d[j0:j1,i0:i1].shape[1]
        obs_mean = np.empty((nvar,nevents,dx,dy),dtype=np.float64)
        obs_err = np.empty((nvar,nevents,dx,dy),dtype=np.float64)
        mod_bias = np.empty((nmod,nvar,nevents,dx,dy),dtype=np.float64)
        mod_exceed = np.empty((nmod,nvar,nevents,dx,dy),dtype=np.float64)

    # Truncate to the period of interest for climate means
    for eind,ename in enumerate(events):

        # Get relevant time indices
        tind = np.asarray([years[x] in ytarget[eind] and months[x] in mtarget[eind] for x in range(len(years))])

        if mind == 0:

            # Calculate the observed mean for each of the variables
            obs_mean[0,eind,:,:] = np.nanmean(obsprecipma[tind,j0:j1,i0:i1],axis=0)
            obs_mean[1,eind,:,:] = np.nanmean(obst2maxma[tind,j0:j1,i0:i1],axis=0)
            obs_mean[2,eind,:,:] = np.nanmean(obst2minma[tind,j0:j1,i0:i1],axis=0)

            # Calculate the observed mean error
            obs_err[0,eind,:,:] = np.nanmean(obspreciprmsma[tind,j0:j1,i0:i1],axis=0)
            obs_err[1,eind,:,:] = np.nanmean(obst2maxrmsma[tind,j0:j1,i0:i1],axis=0)
            obs_err[2,eind,:,:] = np.nanmean(obst2minrmsma[tind,j0:j1,i0:i1],axis=0)
        
        # Get the model bias for each variable
        mod_bias[mind,0,eind,:,:] = np.nanmean(precip[tind,j0:j1,i0:i1],axis=0) - obs_mean[0,eind,:,:]
        mod_bias[mind,1,eind,:,:] = np.nanmean(t2max[tind,j0:j1,i0:i1],axis=0) - obs_mean[1,eind,:,:]
        mod_bias[mind,2,eind,:,:] = np.nanmean(t2min[tind,j0:j1,i0:i1],axis=0) - obs_mean[2,eind,:,:]
    
        # Get the model bias exceedence of the observational uncertainty for each variable
        mod_exceed[mind,:,eind,:,:] = abs(mod_bias[mind,:,eind,:,:]) - obs_err[:,eind,:,:]
    
        del tind
             
    del precip, t2max, t2min
    del time,timesplit,years,months

# Calculate the multi-model range in the bias
mod_range = np.nanmax(mod_bias,axis=0) - np.nanmin(mod_bias,axis=0)

# Calculate the number of members within the observational uncertainty
mod_count = np.nansum(np.where(mod_exceed <= 0, 1, 0),axis=0)/nmod


In [ ]:
# https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph
import matplotlib.pyplot as plt
from mpl_toolkits import axes_grid1

def add_colorbar(im, aspect=20, pad_fraction=0.5, **kwargs):
    """Add a vertical color bar to an image plot."""
    divider = axes_grid1.make_axes_locatable(im.axes)
    width = axes_grid1.axes_size.AxesY(im.axes, aspect=1./aspect)
    pad = axes_grid1.axes_size.Fraction(pad_fraction, width)
    current_ax = plt.gca()
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.sca(current_ax)
    return im.axes.figure.colorbar(im, cax=cax, **kwargs)


In [ ]:
# Function to plot data
def plot_contour(obs_mean,obs_err,mod_bias,mod_count,lsmask,lat,lon,mtitle,figurename):
    """This function plots a comparison between observational data and any number of model experiments.
    
    Input arguments: 

    """
    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    
    plt.rcParams['savefig.dpi'] = 1000
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"

    # How many variables
    nvar = obs_mean.shape[0]
        
    # Create figure object and subplots
    nrow=nvar
    ncol=4

    gs = mpl.gridspec.GridSpec(nrow,ncol, wspace=0.1)
    fig = plt.figure(figsize=(20,12))

    # Make axes
    ax11 = fig.add_subplot(gs[0,0])
    ax12 = fig.add_subplot(gs[0,1])
    ax13 = fig.add_subplot(gs[0,2])
    ax14 = fig.add_subplot(gs[0,3])
    ax21 = fig.add_subplot(gs[1,0])
    ax22 = fig.add_subplot(gs[1,1])
    ax23 = fig.add_subplot(gs[1,2])
    ax24 = fig.add_subplot(gs[1,3])
    ax31 = fig.add_subplot(gs[2,0])
    ax32 = fig.add_subplot(gs[2,1])
    ax33 = fig.add_subplot(gs[2,2])
    ax34 = fig.add_subplot(gs[2,3])

    nbins=12
    
    # Plot OBS mean in the first column

    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=False, ax=ax11)
    mymap.drawcoastlines()    
    ax11.set_title('(a) AGCD MEAN %s' %(mtitle[0]), fontweight='bold')
    ax11.set_ylabel('Latitude')
    ax11.set_xticks([],[])
    c11 = ax11.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,obs_mean[0,:,:]),vmin=0.,vmax=10., # 5., #
                          cmap=plt.get_cmap('YlGn'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins*2).tick_values(0.,10.), ncolors=plt.get_cmap('YlGn').N, clip=True))
    add_colorbar(c11, ax=ax11)#,ticks=[0,1,2,3,4,5])

    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=False, ax=ax21)
    mymap.drawcoastlines()    
    ax21.set_title('(e) AGCD MEAN %s' %(mtitle[1]), fontweight='bold')
    ax21.set_ylabel('Latitude')
    ax21.set_xticks([],[])
    c21 = ax21.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,obs_mean[1,:,:]),vmin=0.,vmax=40.,
                          cmap=plt.get_cmap('plasma_r'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins*2).tick_values(0.,40.), ncolors=plt.get_cmap('plasma_r').N, clip=True))
    add_colorbar(c21, ax=ax21)
    
    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=False, ax=ax31)
    mymap.drawcoastlines()    
    ax31.set_title('(i) AGCD MEAN %s' %(mtitle[2]), fontweight='bold')
    ax31.set_ylabel('Latitude')
    ax31.set_xlabel('Longitude')
    c31 = ax31.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,obs_mean[2,:,:]),vmin=0.,vmax=40.,
                          cmap=plt.get_cmap('plasma_r'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins*2).tick_values(0.,40.), ncolors=plt.get_cmap('plasma_r').N, clip=True))
    add_colorbar(c31, ax=ax31)

    # Plot the OBS error in the second column
    
    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=True, ax=ax12)
    mymap.drawcoastlines()    
    ax12.set_title('(b) AGCD ERROR %s' %(mtitle[0]), fontweight='bold')
    c12 = ax12.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,obs_err[0,:,:]),vmin=0.,vmax=10., #2.5, #
                          cmap=plt.get_cmap('Reds'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins).tick_values(0.,10.), ncolors=plt.get_cmap('Reds').N, clip=True))
    add_colorbar(c12, ax=ax12)

    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=True, ax=ax22)
    mymap.drawcoastlines()    
    ax22.set_title('(f) AGCD ERROR %s' %(mtitle[1]), fontweight='bold')
    c22 = ax22.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,obs_err[1,:,:]),vmin=0.,vmax=5.,
                          cmap=plt.get_cmap('Reds'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins).tick_values(0.,5.), ncolors=plt.get_cmap('Reds').N, clip=True))
    add_colorbar(c22, ax=ax22)
    
    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=False, ax=ax32)
    mymap.drawcoastlines()    
    ax32.set_title('(j) AGCD ERROR %s' %(mtitle[2]), fontweight='bold')
    ax32.set_xlabel('Longitude')
    ax32.set_yticks([],[])
    c32 = ax32.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,obs_err[2,:,:]),vmin=0.,vmax=5.,
                          cmap=plt.get_cmap('Reds'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins).tick_values(0.,5.), ncolors=plt.get_cmap('Reds').N, clip=True))
    add_colorbar(c32, ax=ax32)

    # Plot the Mean Model Bias in the third column
    
    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=True, ax=ax13)
    mymap.drawcoastlines()    
    ax13.set_title('(c) Mean Bias %s' %(mtitle[0]), fontweight='bold')
    c13 = ax13.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,mod_bias[0,:,:]),vmin=-5.,vmax=5., #vmin=-2.5,vmax=2.5, #
                          cmap=plt.get_cmap('BrBG'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins).tick_values(-5.,5.), ncolors=plt.get_cmap('BrBG').N, clip=True))
    add_colorbar(c13, ax=ax13)

    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=True, ax=ax23)
    mymap.drawcoastlines()    
    ax23.set_title('(g) Mean Bias %s' %(mtitle[1]), fontweight='bold')
    c23 = ax23.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,mod_bias[1,:,:]),vmin=-5.,vmax=5.,
                          cmap=plt.get_cmap('seismic'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins).tick_values(-5.,5.), ncolors=plt.get_cmap('seismic').N, clip=True))
    add_colorbar(c23, ax=ax23)
    
    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=False, ax=ax33)
    mymap.drawcoastlines()    
    ax33.set_title('(k) Mean Bias %s' %(mtitle[2]), fontweight='bold')
    ax33.set_xlabel('Longitude')
    ax33.set_yticks([],[])
    c33 = ax33.pcolormesh(lon,lat,np.ma.masked_where(lsmask==0,mod_bias[2,:,:]),vmin=-5.,vmax=5.,
                          cmap=plt.get_cmap('seismic'),
                          norm=BoundaryNorm(MaxNLocator(nbins=nbins).tick_values(-5.,5.), ncolors=plt.get_cmap('seismic').N, clip=True))
    add_colorbar(c33, ax=ax33)
    
    # Plot the model count in the fourth column

    countcmap = plt.get_cmap('RdYlGn')
    countcmap.set_bad('w')
    countlevels = MaxNLocator(nbins=nbins).tick_values(0.,1.)
    countnorm = BoundaryNorm(countlevels, ncolors=countcmap.N, clip=True)
    
    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=True, ax=ax14)
    mymap.drawcoastlines()    
    ax14.set_title('(d) Model Fraction within $Err_{OBS}$', fontweight='bold')
    c14 = ax14.pcolormesh(lon,lat,np.ma.masked_where(mod_count[0,:,:]==0,mod_count[0,:,:]),vmin=0.,vmax=1.,cmap=countcmap,norm=countnorm)
    add_colorbar(c14, ax=ax14)

    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=True, ax=ax24)
    mymap.drawcoastlines()    
    ax24.set_title('(h) Model Fraction within $Err_{OBS}$', fontweight='bold')
    c24 = ax24.pcolormesh(lon,lat,np.ma.masked_where(mod_count[0,:,:]==0,mod_count[1,:,:]),vmin=0.,vmax=1.,cmap=countcmap,norm=countnorm)
    add_colorbar(c24, ax=ax24)
    
    mymap = Basemap(projection='cyl', llcrnrlat=np.nanmin(lat), urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon), urcrnrlon=np.nanmax(lon), resolution='c', suppress_ticks=False, ax=ax34)
    mymap.drawcoastlines()    
    ax34.set_title('(l) Model Fraction within $Err_{OBS}$', fontweight='bold')
    ax34.set_xlabel('Longitude')
    ax34.set_yticks([],[])
    c34 = ax34.pcolormesh(lon,lat,np.ma.masked_where(mod_count[0,:,:]==0,mod_count[2,:,:]),vmin=0.,vmax=1.,cmap=countcmap,norm=countnorm)
    add_colorbar(c34, ax=ax34)
    
    fig.subplots_adjust(wspace=0.005, hspace=0.05)
    fig.savefig(figurename)
    plt.close(fig)


In [ ]:
# Loop through the different events
vlabels=['PR [mm $day{^-1}$]','TX [\xb0 C]','TN [\xb0 C]']
for eind,ename in enumerate(events):
    figurename='%s/%s_%s.png' %(fig_dir,fig_name_prefix,events[eind])
    plot_contour(obs_mean[:,eind,:,:],obs_err[:,eind,:,:],np.nanmean(mod_bias[:,:,eind,:,:],axis=0),mod_count[:,eind,:,:],lsmask[j0:j1,i0:i1],lat2d[j0:j1,i0:i1],lon2d[j0:j1,i0:i1],vlabels,figurename)
   